<a href="https://colab.research.google.com/github/gptix/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/Jud_Taylor_assignment_regression_classification_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


# Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?



## Prepare import path based on environment.

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

## Import useful libraries.

In [0]:
# Import useful libraries.
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler

## Load raw data.

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
raw_df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

## Make a working copy of raw data.

In [0]:
# Make a working copy of the raw df.
df = raw_df

## Review dataframe.

In [152]:
# Review dataframe.
# df
# df.describe()
# df.describe().T
# df.head()
# df.tail()
# df.columns
# df.dtypes
df.shape
# df.columns.isna()
# df.isna()

(423, 66)

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.



## Engineer data.

In [0]:
# Replace spaces in column names to underbars
df.columns = [col.replace(' ', '_') for col in df]
# df.columns
# df.head()

In [0]:
# Convert ingredients columns to one-hot encodings
ingredients = ['Beef','Pico','Guac','Cheese','Fries','Sour_cream','Pork','Chicken',
               'Shrimp','Fish','Rice','Beans','Lettuce','Tomato','Bell_peper',
               'Carrots','Cabbage','Sauce','Salsa.1','Cilantro','Onion',
               'Taquito','Pineapple','Ham','Chile_relleno','Nopales','Lobster',
               'Queso','Egg','Mushroom','Bacon','Sushi','Avocado','Corn',
               'Zucchini']

def check_uniques():
  '''List unique values in columns.'''
  for i in ingredients:
    print(i + str(df[i].unique()))     

In [0]:
check_uniques()

In [0]:
# Replace non-zeroes in columns with ones  
for i in ingredients:
  df[i] = df[i].map({'x': 1, 'X': 1})

check_uniques()

In [0]:
# Replace NaN's with 0
for i in ingredients:
  df[i] = df[i].fillna(0)
check_uniques()

In [0]:
# We will use an 'overall' value as 'Great', which will be encoded as True/1
# Drop unrated burritos.
df = df.dropna(subset=['overall'])

# Add a feature to hold TF for 'Great'
df['Great'] = df['overall'] >= 4

In [0]:
# df.Great

In [164]:
# Clean/combine the Burrito categories

# make all text in 'Burrito' column lowercase, to simplify manipulaton.
df['Burrito'] = df['Burrito'].str.lower()

# define predicates (Y/N tests)
california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

# Apply predicates, and apply catchall for remainder
df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [166]:
# df.Burrito

0      California
1      California
2        Carnitas
3           Asada
4      California
          ...    
418         Other
419         Other
420    California
421         Other
422         Other
Name: Burrito, Length: 421, dtype: object

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

## Split into Train, Validate, Test sets
- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.

In [171]:
# Create a field to hold year.
df['Year'] = pd.DatetimeIndex(  pd.to_datetime(df['Date'], format='%m/%d/%Y')  ).year
df['Year']

0      2016
1      2016
2      2016
3      2016
4      2016
       ... 
418    2019
419    2019
420    2019
421    2019
422    2019
Name: Year, Length: 421, dtype: int64

In [180]:
train = df[df['Year'] <= 2016]
val = df[df['Year'] == 2017]
test = df[df['Year'] >= 2018]

shapes = train.shape, val.shape, test.shape
shapes

((298, 60), (85, 60), (38, 60))

## Baseline classification
- [ ] Begin with baselines for classification.

In [201]:
target = 'Great'
y_train = train[target]
print(y_train.shape)
split_train = y_train.value_counts(normalize=True)
great_percentage_train = split_train[1]*100
baseline = y_train.mode().values[0]
print("Baseline: is it great?  - " + str(baseline))
print("Percentage of great burritos: " + str(round(great_percentage_train, 2)))

(298,)
Baseline: is it great?  - False
Percentage of great burritos: 40.94


In [202]:
# Test baseline against val set
y_val = val[target]
print(y_val.shape)
split_val = y_val.value_counts(normalize=True)
# y_val.value_counts(normalize=True)

great_percentage_val = split_val[1]*100
print("Percentage of great burritos: " + str(round(great_percentage_val, 2)))
# print("Difference between ")


(85,)
Percentage of great burritos: 44.71


## Logistic Regression
- [ ] Use scikit-learn for logistic regression.

In [185]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = 'Survived'
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]




log_reg = LogisticRegression(solver='lbfgs')
log_reg.fit(X_train, y_train)
print('Validation Accuracy', log_reg.score(X_val_imputed, y_val))

NameError: ignored

- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).